<a href="https://colab.research.google.com/github/marco-siino/DA-ESWA/blob/main/code/augmentation/HSS_TestSet_Augmentation_IT_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing modules.

In [1]:
import os
import os.path
from os import path
import re
import shutil
import string
import tensorflow as tf

!pip install -U deep-translator
from deep_translator import GoogleTranslator
from io import open
from pathlib import Path
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 2.9 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
!pip install xmltodict
import xmltodict 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing DS and extract in current working directory.

In [3]:
# Url obtained starting from this: https://drive.google.com/file/d/19ZcqEv88euKB71HfAWjTGN3uCKp2qsfP/ and forcing export=download.
train_set_url = 'https://github.com/marco-siino/DA-ESWA/raw/main/data/hss/hss-test-cleaned.zip'  # train FNS
train_set_path = tf.keras.utils.get_file("pan21-author-profiling-test-without-gold.zip", train_set_url,
                                    extract=True, archive_format='zip',cache_dir='.',
                                    cache_subdir='')

train_set_dir = os.path.join(os.path.dirname(train_set_path), 'pan21-author-profiling-test-without-gold.zip')

print(train_set_path)
print(train_set_dir)

652227/652227 [==============================] - 0s 0us/step
./pan21-author-profiling-test-without-gold.zip
./pan21-author-profiling-test-without-gold.zip


## Function to pre-process source text.

In [4]:
def custom_standardization(text):
  tag_open_CDATA_removed = re.sub('<\!\[CDATA\[', ' ',text)
  tag_closed_CDATA_removed = re.sub('\]{1,}>', ' ',tag_open_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en">', ' ',tag_closed_CDATA_removed)
  tag_author_lang_en_removed = re.sub('<author lang="en"', ' ',tag_closed_CDATA_removed)
  #only for test remove if needed
  tag_author_lang_en_removed = re.sub('class="0">', ' ', tag_author_lang_en_removed )
  tag_author_lang_en_removed = re.sub('class="1">', ' ', tag_author_lang_en_removed )
  ######
  tag_closed_author_removed = re.sub('</author>', '', tag_author_lang_en_removed)
  tag_closed_author_removed = re.sub('</author', '', tag_closed_author_removed)
  tag_open_documents_removed = re.sub('<documents>\n(\t){0,2}', '',tag_closed_author_removed)
  tag_closed_documents_removed = re.sub('</documents>\n(\t){0,2}', ' ',tag_open_documents_removed)
  lowercased_text = tag_closed_documents_removed.lower()
  return lowercased_text

## Data Augmentation function.

In [5]:
def chunkstring(string, length):
  res = list((string[0+i:length+i] for i in range(0, len(string), length)))
  return res

def enhance_one_sample(sample, TARGET='it', return_both=True):  
  preprocessed_text = custom_standardization(sample)

  #chunk to avoid character limits  
  TOBETRANS = chunkstring(preprocessed_text, 4000)
  translated_it = GoogleTranslator(source='en', target=TARGET).translate_batch(TOBETRANS)
  reversed_trans = GoogleTranslator(source=TARGET, target='en').translate_batch(translated_it)
  merged_chunks =''.join(reversed_trans)
  enhanced_sample = preprocessed_text+merged_chunks
  if return_both == False:
    enhanced_sample = merged_chunks
  return enhanced_sample

# Generating augmented train set.

In [6]:
## Generate the augmented training set as a zip file.

# pan22-author-profiling-training-2022-03-29-augmented.zip has to be created.
if not os.path.exists('pan21-author-profiling-test-without-gold-augmented'):
    os.makedirs('pan21-author-profiling-test-without-gold-augmented')
if not os.path.exists('pan21-author-profiling-test-without-gold-augmented/en/'):
    os.makedirs('pan21-author-profiling-test-without-gold-augmented/en/')

In [7]:
# Copy the ground truth file.
if os.path.exists('/content/pan21-author-profiling-test-without-gold'):
        shutil.copyfile('/content/pan21-author-profiling-test-without-gold/truth.txt', '/content/pan21-author-profiling-test-without-gold-augmented/truth.txt')

In [8]:
file_nr = 0
count_errors = 0
for filename in os.listdir('pan21-author-profiling-test-without-gold/en/'):
  file_nr += 1
  x = filename.split(".")      
  author_id = x[0]
  print("File nr.:", file_nr)
  print("Filename:", filename)
  text = open('pan21-author-profiling-test-without-gold/en/'+filename, 'r').read()
  try:
    enhanced_sample = enhance_one_sample(text, TARGET='it')
    #de_back = enhance_one_sample(text, TARGET='de', return_both=False)
    #enhanced_sample = enhanced_sample + de_back    
    f = open("pan21-author-profiling-test-without-gold-augmented/en/"+author_id+".xml", "a")
    f.write(enhanced_sample)
    f.close()
    print('Succes')
  except:
    print('!!! FAILED !!!!')
    count_errors += 1
    f = open("pan21-author-profiling-test-without-gold-augmented/en/"+author_id+".xml", "a")
    preprocessed_text = custom_standardization(text)
    f.write(preprocessed_text)
    f.close()

print('cunt err', count_errors)
!zip -r pan21-author-profiling-test-without-gold-augmented.zip pan21-author-profiling-test-without-gold-augmented
# If automatic download doesn't start, open the directory browser on the left menu and download the zip file manually.
files.download("pan21-author-profiling-test-without-gold-augmented.zip")

File nr.: 1
Filename: 5f57628b5bddcbf35fec6cbca21da8ef.xml
Succes
File nr.: 2
Filename: 5724d92d5cd1a777db48f68e679a94ca.xml
Succes
File nr.: 3
Filename: 07275a64a6a197f348253cd98bf6ddc5.xml
Succes
File nr.: 4
Filename: f8e9b6073d2fcbf9d2de3df07c62d86e.xml
Succes
File nr.: 5
Filename: 14c19eb78928560a39963b0d3c7b6730.xml
Succes
File nr.: 6
Filename: 7610534fcf5bf40d6e4b4073f2b980b4.xml
Succes
File nr.: 7
Filename: dbcb2f9a069bd686b58161ddaa5529ce.xml
Succes
File nr.: 8
Filename: 8b46f62667ec355d99f12f91eb5c1fb1.xml
Succes
File nr.: 9
Filename: 6838d5b68f5e6d0aab4c492de5002614.xml
Succes
File nr.: 10
Filename: 061a343a9faec7d73cedd1aedc59b07c.xml
Succes
File nr.: 11
Filename: 8e785d3cef887ca5f5a308edbb466519.xml
Succes
File nr.: 12
Filename: 5ae45806800bbc1c378793eca6af14c9.xml
Succes
File nr.: 13
Filename: 3348345aac7462125d72b02c3a19fe3d.xml
Succes
File nr.: 14
Filename: 05e1527e1d7210f6da3a90d4c7e53dd7.xml
Succes
File nr.: 15
Filename: fec7f040a4100395d78d1a94a148e956.xml
Succes
File

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>